In [ ]:
import argparse
import requests
from bs4 import BeautifulSoup
import csv
import time
from typing import Dict

def parse_university(url: str) -> Dict[str, str]:#ключики и значения на выходе будут строками
    #настройка, чтоб нас не считали за бота
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    #пытаемся пройти по url из файлика
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f'site error {url}: {e}')
        return None

    # Здесь движемся в html в контейнер за названием универа
    name = soup.select_one('#__next > main > div > div > div > div > div.css-1dwnmlj > div.desktopHeaderRef.css-wxa6cs > div.css-17ek16l > div > div > div.css-axsv8p > div:nth-child(1) > h1')
    univer_name = name.text.strip() if name else 'Без названия'

    #вот такие столбцы здесь будут
    data = {
        'Name': univer_name,
        'Arts and Humanities': '',
        'Business and Economics': '',
        'Computer Science': '',
        'Education Studies': '',
        'Engineering': '',
        'Law': '',
        'Life Sciences': '',
        'Medical and Health': '',
        'Physical Sciences': '',
        'Psychology': '',
        'Social Sciences': ''
    }

    #идём по вкладкам с предметами, начиная с контейнера
    sub = soup.select_one('#subjects > div.css-eoiiov')


    #categ_name - имя категории
    #sub - контент с предметами
    #name_el - имя элемента
    #sub_list - перечень предметов
    if sub:
        for categ_div in sub.find_all('div', recursive=False):
            name_el = categ_div.select_one('div > h3')
            #глядим в каждый див и парсим оттуда заголовки, чтоб потом сверить с столбцами 
            if name_el:
                categ_name = name_el.text.strip()
                #здесь заглядываем в список всех предметов и идём по каждому элементику и достаём в одну ячейку
                if categ_name in data:
                    sub_list = categ_div.select_one('div > ul')
                    if sub_list:
                        subjects = []
                        for li in sub_list.find_all('li'):
                            subject = li.text.strip()
                            if subject:
                                subjects.append(subject)
                        
                        data[categ_name] = ", ".join(subjects) #мёрджим через запятую в одну ячейку

    return data

def main():#для параллельного запуска прикольно сделать стоп и старт индексы
    parser = argparse.ArgumentParser(description='парсим')
    parser.add_argument('--start', type=int, default=1, help='с индекса ')# в default меняем цифры
    parser.add_argument('--end', type=int, default=2, help='до индекса ')#в default меняем цифры
    parser.add_argument('--input', type=str, default='the_rankings_2026_full.csv', help='входной файл (надеюсь он у вас лежит в папке с кодом))')
    parser.add_argument('--output', type=str, default='universities_subjects12.csv', help='Вот csv-файл на выходе')
    #для разных промежутков лучше менять вот здесь цифры 'universities_subjects{2500_2811}.csv' на цифры из default начала и конца
    
    args = parser.parse_args()
    
    input_f = args.input
    output_f = args.output
    start_i = args.start
    end_i = args.end
    
    #в файле берём ссылки и идём по ним на страницу каждого универа
    urls = []
    try:
        with open(input_f, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if 'URL' in row and row['URL']:
                    urls.append(row['URL'])
    except FileNotFoundError:
        print(f"нет файла {input_f}")
        return
    
    #регулируем конечный индекс
    if end_i > len(urls):
        end_i = len(urls)
    
    select = urls[start_i:end_i]#выбираем все ссылки по индексам
    
    #парсим вузы
    all_data = []
    for i, url in enumerate(select, start=1):
        #чекаем какая запись сейчас идёт
        print(f"сейчас {i}/{len(select)}: {url}")
        
        data = parse_university(url)
        if data:
            all_data.append(data)
        time.sleep(1)  #чтобы быть менее похожим на бота
    
    #передаём в csv
    if all_data:
        fieldnames = [
            'Name', 'Arts and Humanities', 'Business and Economics', 
            'Computer Science', 'Education Studies', 'Engineering', 
            'Law', 'Life Sciences', 'Medical and Health', 
            'Physical Sciences', 'Psychology', 'Social Sciences'
        ]
        
        with open(output_f, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_data)
        
        print(f"\nзагрузили в '{output_f}'")
    else:
        print('всё пусто, попробуйте другие индексы')

if __name__ == '__main__':
    main()
